In [5]:
import oss2
import json
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import requests

with open("../aliyun_key.json" ,'r') as key:
    AccessKeyDict=json.loads(key.read())
AccessKeyId=AccessKeyDict["AccessKeyId"]
AccessKeySecret=AccessKeyDict["AccessKeySecret"]
auth = oss2.Auth(AccessKeyId, AccessKeySecret)
bucket = oss2.Bucket(auth, 'https://oss-cn-shanghai.aliyuncs.com', 'ocr-yjs')
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkcore.auth.credentials import AccessKeyCredential
from aliyunsdkcore.auth.credentials import StsTokenCredential
credentials = AccessKeyCredential(AccessKeyId, AccessKeySecret)
client = AcsClient(region_id='cn-shanghai', credential=credentials)
import seaborn as sns

In [13]:
"""
https://help.aliyun.com/document_detail/164973.html
五官编辑
五官编辑能力可以对人脸的五官进行编辑并返回编辑后的结果图片。例如用户输入目标人脸图、参考人脸图、需要编辑的五官（鼻子、嘴巴、下巴、眼睛、眉毛），
接口会将参考人脸图对应的五官替换融合到目标人脸图，最后返回替换融合后的结果图。
"""

from aliyunsdkfacebody.request.v20191230.SwapFacialFeaturesRequest import SwapFacialFeaturesRequest

request = SwapFacialFeaturesRequest()
request.set_accept_format('json')

filepath="images/test2.jpg"


def get_url(filepath):
    with open(filepath, 'rb') as fileobj:
        bucket.put_object(filepath, fileobj)
    url="https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/"+filepath
    return url

url=get_url(filepath)   
print(url)

SourceImageURL=get_url("images/SourceImageURL.jpg")
request.set_EditPart("eyes")
request.set_TargetImageURL(url) # 参考人脸图像
request.set_SourceImageURL(SourceImageURL)

response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)

https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/images/test2.jpg
{'RequestId': 'F7A9A1C7-69FE-5567-8F15-EE7711A8B4D8', 'Data': {'ImageURL': 'http://vibktprfx-prod-prod-aic-gd-cn-shanghai.oss-cn-shanghai.aliyuncs.com/face-edit/2303402f-14c6-4480-a47a-c5ef1d0d63b7__710d20210821-011732.jpg?Expires=1629510453&OSSAccessKeyId=LTAI4FoLmvQ9urWXgSRpDvh1&Signature=UxrVh32hrCtRCpng%2B1LJtYIEqZE%3D'}}


In [14]:
ImageURL=response["Data"]["ImageURL"]

ImageURL

'http://vibktprfx-prod-prod-aic-gd-cn-shanghai.oss-cn-shanghai.aliyuncs.com/face-edit/2303402f-14c6-4480-a47a-c5ef1d0d63b7__710d20210821-011732.jpg?Expires=1629510453&OSSAccessKeyId=LTAI4FoLmvQ9urWXgSRpDvh1&Signature=UxrVh32hrCtRCpng%2B1LJtYIEqZE%3D'

In [16]:
"""
SourceImageURL	String	否	https://viapi-test.oss-cn-shanghai.aliyuncs.com/test-team/facialSource.jpg	
目标人脸图像URL地址。

TargetImageURL	String	否	https://viapi-test.oss-cn-shanghai.aliyuncs.com/test-team/facialtarget.jpg	
参考人脸图像URL地址。接口会将该参考图中的五官内容替换融合到目标人脸图像中。
"""

'\nSharp\tFloat\t是\t1\t\n锐化（清晰度）程度，取值范围0~1。越接近1锐化程度越高。\n\nSmooth\tFloat\t是\t0.1\t\n平滑程度，取值范围0~1。越接近1平滑程度越高。\n\nWhite\tFloat\t是\t0.3\t\n美白程度，取值范围0~1。越接近1美白程度越高。\n'

In [15]:
with open("images/五官编辑后.jpg",'wb') as file:
    file.write(requests.get(ImageURL).content)